# Utaratibu wa Binadamu-Katika-Mzunguko na Mfumo wa Wakala wa Microsoft

## 🎯 Malengo ya Kujifunza

Katika daftari hili, utajifunza jinsi ya kutekeleza utaratibu wa **binadamu-katika-mzunguko** kwa kutumia `RequestInfoExecutor` ya Mfumo wa Wakala wa Microsoft. Muundo huu wenye nguvu unaruhusu kusimamisha mzunguko wa AI ili kukusanya maoni ya binadamu, na kufanya mawakala wako kuwa wa maingiliano huku ukitoa udhibiti kwa binadamu juu ya maamuzi muhimu.

## 🔄 Binadamu-Katika-Mzunguko ni Nini?

**Binadamu-katika-mzunguko (HITL)** ni muundo wa kubuni ambapo mawakala wa AI husimamisha utekelezaji ili kuomba maoni ya binadamu kabla ya kuendelea. Hii ni muhimu kwa:

- ✅ **Maamuzi muhimu** - Pata idhini ya binadamu kabla ya kuchukua hatua muhimu
- ✅ **Hali zisizoeleweka** - Waache binadamu kufafanua pale AI inapokuwa na mashaka
- ✅ **Mapendeleo ya mtumiaji** - Waulize watumiaji kuchagua kati ya chaguo nyingi
- ✅ **Uzingatiaji & usalama** - Hakikisha usimamizi wa binadamu kwa shughuli zinazodhibitiwa
- ✅ **Uzoefu wa maingiliano** - Unda mawakala wa mazungumzo wanaojibu maoni ya mtumiaji

## 🏗️ Jinsi Inavyofanya Kazi katika Mfumo wa Wakala wa Microsoft

Mfumo huu unatoa vipengele vitatu muhimu kwa HITL:

1. **`RequestInfoExecutor`** - Mtekelezaji maalum anayesimamisha mzunguko na kutoa tukio la `RequestInfoEvent`
2. **`RequestInfoMessage`** - Darasa la msingi kwa ujumbe wa maombi uliopangwa kutumwa kwa binadamu
3. **`RequestResponse`** - Inahusisha majibu ya binadamu na maombi ya awali kwa kutumia `request_id`

**Muundo wa Mzunguko:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Mfano Wetu: Kuhifadhi Hoteli kwa Uthibitisho wa Mtumiaji

Tutajenga juu ya mzunguko wa masharti kwa kuongeza uthibitisho wa binadamu **kabla** ya kupendekeza maeneo mbadala:

1. Mtumiaji anaomba eneo (mfano, "Paris")
2. `availability_agent` hukagua kama vyumba vinapatikana
3. **Ikiwa hakuna vyumba** → `confirmation_agent` huuliza "Je, ungependa kuona mbadala?"
4. Mzunguko **unasimama** kwa kutumia `RequestInfoExecutor`
5. **Binadamu anajibu** "ndiyo" au "hapana" kupitia pembejeo ya console
6. `decision_manager` huamua kulingana na jibu:
   - **Ndiyo** → Onyesha maeneo mbadala
   - **Hapana** → Futa ombi la kuhifadhi
7. Onyesha matokeo ya mwisho

Hii inaonyesha jinsi ya kuwapa watumiaji udhibiti juu ya mapendekezo ya wakala!

---

Tuanzie sasa! 🚀


## Hatua ya 1: Ingiza Maktaba Zinazohitajika

Tunajumuisha vipengele vya kawaida vya Agent Framework pamoja na **madarasa maalum ya binadamu-katika-mzunguko**:
- `RequestInfoExecutor` - Mtekelezaji anayesimamisha mtiririko wa kazi kwa ajili ya maoni ya binadamu
- `RequestInfoEvent` - Tukio linalotolewa wakati maoni ya binadamu yanahitajika
- `RequestInfoMessage` - Darasa la msingi kwa data ya ombi iliyoainishwa
- `RequestResponse` - Inahusisha majibu ya binadamu na maombi
- `WorkflowOutputEvent` - Tukio la kugundua matokeo ya mtiririko wa kazi


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Hatua ya 2: Fafanua Miundo ya Pydantic kwa Matokeo Yaliyopangwa

Miundo hii hufafanua **schema** ambayo mawakala watarudisha. Tunahifadhi miundo yote kutoka kwa mtiririko wa kazi wa masharti na kuongeza:

**Mpya kwa Mchakato wa Binadamu:**
- `HumanFeedbackRequest` - Darasa ndogo la `RequestInfoMessage` linalofafanua mzigo wa ombi unaotumwa kwa binadamu  
  - Inajumuisha `prompt` (swali la kuuliza) na `destination` (muktadha kuhusu mji usiopatikana)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Hatua ya 3: Tengeneza Zana ya Kuhifadhi Hoteli

Zana ile ile kutoka kwa mtiririko wa kazi wa masharti - inakagua kama vyumba vinapatikana katika eneo husika.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Hatua ya 4: Kufafanua Kazi za Masharti kwa Usambazaji

Tunahitaji **kazi nne za masharti** kwa mtiririko wa kazi unaohusisha binadamu:

**Kutoka kwa mtiririko wa kazi wa masharti:**
1. `has_availability_condition` - Inasambaza wakati hoteli ZINAPATIKANA
2. `no_availability_condition` - Inasambaza wakati hoteli HAZIPATIKANI

**Mpya kwa mtiririko wa kazi unaohusisha binadamu:**
3. `user_wants_alternatives_condition` - Inasambaza wakati mtumiaji anasema "ndiyo" kwa mbadala
4. `user_declines_alternatives_condition` - Inasambaza wakati mtumiaji anasema "hapana" kwa mbadala


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Hatua ya 5: Unda Mtekelezaji wa Decision Manager

Hii ndiyo **kiini cha muundo wa binadamu-katika-mzunguko**! `DecisionManager` ni `Executor` maalum ambayo:

1. **Inapokea maoni ya binadamu** kupitia vitu vya `RequestResponse`
2. **Inachakata uamuzi wa mtumiaji** (ndiyo/hapana)
3. **Inaelekeza mtiririko wa kazi** kwa kutuma ujumbe kwa mawakala husika

Vipengele muhimu:
- Inatumia mapambo ya `@handler` kufichua mbinu kama hatua za mtiririko wa kazi
- Inapokea `RequestResponse[HumanFeedbackRequest, str]` inayojumuisha ombi la awali na jibu la mtumiaji
- Inatoa ujumbe rahisi wa "ndiyo" au "hapana" ambao husababisha kazi za hali yetu


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Hatua ya 6: Unda Mtekelezaji wa Maonyesho Maalum

Mtekelezaji wa maonyesho sawa kutoka kwa mtiririko wa kazi wa masharti - hutoa matokeo ya mwisho kama matokeo ya mtiririko wa kazi.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Hatua ya 7: Pakia Vigezo vya Mazingira

Sanidi mteja wa LLM (GitHub Models, Azure OpenAI, au OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Hatua ya 8: Unda Mawakala wa AI na Watekelezaji

Tunaunda **vijenzi sita vya mtiririko wa kazi**:

**Mawakala (wamefungwa ndani ya AgentExecutor):**
1. **availability_agent** - Hukagua upatikanaji wa hoteli kwa kutumia zana
2. **confirmation_agent** - 🆕 Huandaa ombi la uthibitisho wa binadamu
3. **alternative_agent** - Hupendekeza miji mbadala (wakati mtumiaji anasema ndiyo)
4. **booking_agent** - Huhamasisha uhifadhi (wakati vyumba vinapatikana)
5. **cancellation_agent** - 🆕 Hushughulikia ujumbe wa kughairi (wakati mtumiaji anasema hapana)

**Watekelezaji Maalum:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` inayosimamisha mtiririko wa kazi kwa maoni ya binadamu
7. **decision_manager** - 🆕 Mtekelezaji maalum anayelekeza kulingana na jibu la binadamu (tayari imefafanuliwa hapo juu)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Hatua ya 9: Jenga Mtiririko wa Kazi na Mtu Kati

Sasa tunaunda grafu ya mtiririko wa kazi na **mwelekeo wa masharti** ikijumuisha njia ya mtu kati:

**Muundo wa Mtiririko wa Kazi:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Viungo Muhimu:**
- `availability_agent → confirmation_agent` (wakati hakuna vyumba)
- `confirmation_agent → prepare_human_request` (badilisha aina)
- `prepare_human_request → request_info_executor` (simama kwa mtu)
- `request_info_executor → decision_manager` (daima - hutoa RequestResponse)
- `decision_manager → alternative_agent` (wakati mtumiaji anasema "ndiyo")
- `decision_manager → cancellation_agent` (wakati mtumiaji anasema "hapana")
- `availability_agent → booking_agent` (wakati vyumba vinapatikana)
- Njia zote zinaishia kwa `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Hatua ya 10: Endesha Jaribio la Kwanza - Jiji BILA Upatikanaji (Paris na Uthibitisho wa Binadamu)

Jaribio hili linaonyesha **mzunguko kamili wa binadamu katika mchakato**:

1. Omba hoteli huko Paris
2. availability_agent inakagua → Hakuna vyumba
3. confirmation_agent inaunda swali linaloelekezwa kwa binadamu
4. request_info_executor **inasimamisha mchakato** na kutoa `RequestInfoEvent`
5. **Programu inatambua tukio na kumwuliza mtumiaji kupitia console**
6. Mtumiaji anaandika "ndiyo" au "hapana"
7. Programu inatuma jibu kupitia `send_responses_streaming()`
8. decision_manager inaelekeza kulingana na jibu
9. Matokeo ya mwisho yanaonyeshwa

**Mfumo Muhimu:**
- Tumia `workflow.run_stream()` kwa mzunguko wa kwanza
- Tumia `workflow.send_responses_streaming(pending_responses)` kwa mizunguko inayofuata
- Sikiliza `RequestInfoEvent` kutambua wakati maoni ya binadamu yanahitajika
- Sikiliza `WorkflowOutputEvent` kukamata matokeo ya mwisho


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Hatua ya 11: Endesha Kesi ya Mtihani 2 - Jiji LENYE Upatikanaji (Stockholm - Hakuna Msaada wa Binadamu Unahitajika)

Mtihani huu unaonyesha **njia ya moja kwa moja** wakati vyumba vinapatikana:

1. Omba hoteli huko Stockholm
2. availability_agent inakagua → Vyumba vinapatikana ✅
3. booking_agent inapendekeza uhifadhi
4. display_result inaonyesha uthibitisho
5. **Hakuna msaada wa binadamu unahitajika!**

Mtiririko huu unazunguka kabisa njia inayohusisha msaada wa binadamu wakati vyumba vinapatikana.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Mambo Muhimu na Mazoea Bora ya Binadamu-Katika-Mzunguko

### ✅ Ulichojifunza:

#### 1. **Mfumo wa RequestInfoExecutor**
Mfumo wa binadamu-katika-mzunguko katika Microsoft Agent Framework unatumia vipengele vitatu muhimu:
- `RequestInfoExecutor` - Husimamisha mtiririko wa kazi na kutoa matukio
- `RequestInfoMessage` - Darasa la msingi kwa maombi ya aina (unda darasa ndogo!)
- `RequestResponse` - Huhusisha majibu ya binadamu na maombi ya awali

**Uelewa Muhimu:**
- `RequestInfoExecutor` HAUKUSANYI maoni yenyewe - husimamisha tu mtiririko wa kazi
- Nambari ya programu yako lazima isikilize `RequestInfoEvent` na kukusanya maoni
- Lazima uitishe `send_responses_streaming()` na kamusi inayohusisha `request_id` na jibu la mtumiaji

#### 2. **Mfumo wa Utendaji wa Kutiririsha**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Usanifu Unaotegemea Matukio**
Sikiliza matukio maalum ili kudhibiti mtiririko wa kazi:
- `RequestInfoEvent` - Maoni ya binadamu yanahitajika (mtiririko wa kazi umesimamishwa)
- `WorkflowOutputEvent` - Matokeo ya mwisho yanapatikana (mtiririko wa kazi umekamilika)
- `WorkflowStatusEvent` - Mabadiliko ya hali (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, nk.)

#### 4. **Watendaji Maalum na @handler**
`DecisionManager` inaonyesha jinsi ya kuunda watendaji ambao:
- Hutumia mapambo ya `@handler` kufichua mbinu kama hatua za mtiririko wa kazi
- Kupokea ujumbe wa aina (mfano, `RequestResponse[HumanFeedbackRequest, str]`)
- Kuongoza mtiririko wa kazi kwa kutuma ujumbe kwa watendaji wengine
- Kupata muktadha kupitia `WorkflowContext`

#### 5. **Njia ya Kuelekeza kwa Maamuzi ya Binadamu**
Unaweza kuunda kazi za hali zinazotathmini majibu ya binadamu:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Matumizi Halisi:

1. **Mtiririko wa Idhini**
   - Pata idhini ya meneja kabla ya kushughulikia ripoti za gharama
   - Hitaji ukaguzi wa binadamu kabla ya kutuma barua pepe za kiotomatiki
   - Thibitisha miamala ya thamani kubwa kabla ya utekelezaji

2. **Uchujaji wa Maudhui**
   - Tambua maudhui yenye shaka kwa ukaguzi wa binadamu
   - Waulize wasimamizi kufanya uamuzi wa mwisho kwa hali za ukingo
   - Peleka kwa binadamu pale ambapo ujasiri wa AI ni mdogo

3. **Huduma kwa Wateja**
   - Ruhusu AI kushughulikia maswali ya kawaida kiotomatiki
   - Peleka masuala magumu kwa mawakala wa binadamu
   - Waulize wateja kama wanataka kuzungumza na binadamu

4. **Usindikaji wa Data**
   - Waulize binadamu kutatua maingizo ya data yenye utata
   - Thibitisha tafsiri za AI za nyaraka zisizoeleweka
   - Waache watumiaji kuchagua kati ya tafsiri nyingi sahihi

5. **Mifumo Muhimu kwa Usalama**
   - Hitaji uthibitisho wa binadamu kabla ya hatua zisizoweza kubadilishwa
   - Pata idhini kabla ya kufikia data nyeti
   - Thibitisha maamuzi katika sekta zinazodhibitiwa (huduma za afya, fedha)

6. **Wakala wa Kuingiliana**
   - Unda roboti za mazungumzo zinazouliza maswali ya kufuatilia
   - Tengeneza wizards zinazowaongoza watumiaji kupitia michakato changamano
   - Buni mawakala wanaoshirikiana na binadamu hatua kwa hatua

### 🔄 Ulinganisho: Na vs Bila Binadamu-Katika-Mzunguko

| Kipengele | Mtiririko wa Hali | Mtiririko wa Binadamu-Katika-Mzunguko |
|-----------|-------------------|---------------------------------------|
| **Utendaji** | `workflow.run()` moja | Mzunguko na `run_stream()` + `send_responses_streaming()` |
| **Maoni ya Mtumiaji** | Hakuna (kiotomatiki kabisa) | Maoni ya kuingiliana kupitia `input()` au UI |
| **Vipengele** | Mawakala + Watendaji | + RequestInfoExecutor + DecisionManager |
| **Matukio** | AgentExecutorResponse pekee | RequestInfoEvent, WorkflowOutputEvent, nk. |
| **Kusimamisha** | Hakuna kusimamisha | Mtiririko wa kazi husimama kwa RequestInfoExecutor |
| **Udhibiti wa Binadamu** | Hakuna udhibiti wa binadamu | Binadamu hufanya maamuzi muhimu |
| **Matumizi** | Kiotomatiki | Ushirikiano na usimamizi |

### 🚀 Mifumo ya Juu:

#### Sehemu Nyingi za Maamuzi ya Binadamu
Unaweza kuwa na nodi nyingi za `RequestInfoExecutor` katika mtiririko wa kazi mmoja:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Kushughulikia Muda wa Kusubiri
Tekeleza muda wa kusubiri kwa majibu ya binadamu:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Muunganisho Tajiri wa UI
Badala ya `input()`, unganisha na UI ya wavuti, Slack, Teams, nk.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Binadamu-Katika-Mzunguko wa Hali
Omba maoni ya binadamu tu katika hali maalum:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Mazoea Bora:

1. **Daima Unda Darasa Ndogo la RequestInfoMessage**
   - Hutoa usalama wa aina na uthibitishaji
   - Huwezesha muktadha tajiri kwa uonyeshaji wa UI
   - Hufafanua nia ya kila aina ya ombi

2. **Tumia Maelezo ya Maombi**
   - Jumuisha muktadha kuhusu unachouliza
   - Eleza matokeo ya kila chaguo
   - Weka maswali rahisi na wazi

3. **Shughulikia Maoni Yasiyotarajiwa**
   - Thibitisha majibu ya mtumiaji
   - Toa chaguo-msingi kwa maoni yasiyo sahihi
   - Toa ujumbe wa makosa wazi

4. **Fuatilia Vitambulisho vya Maombi**
   - Tumia uhusiano kati ya request_id na majibu
   - Usijaribu kudhibiti hali mwenyewe

5. **Buni kwa Kutoblock**
   - Usizuie nyuzi ukisubiri maoni
   - Tumia mifumo ya async kote
   - Saidia matukio ya mtiririko wa kazi yanayofanana

### 📚 Dhana Zinazohusiana:

- **Agent Middleware** - Zuia miito ya wakala (daftari la awali)
- **Usimamizi wa Hali ya Mtiririko wa Kazi** - Hifadhi hali ya mtiririko wa kazi kati ya mizunguko
- **Ushirikiano wa Mawakala Wengi** - Changanya binadamu-katika-mzunguko na timu za mawakala
- **Usanifu Unaotegemea Matukio** - Jenga mifumo inayojibu matukio

---

### 🎓 Hongera!

Umebobea katika mtiririko wa kazi wa binadamu-katika-mzunguko na Microsoft Agent Framework! Sasa unajua jinsi ya:
- ✅ Kusimamisha mtiririko wa kazi ili kukusanya maoni ya binadamu
- ✅ Kutumia RequestInfoExecutor na RequestInfoMessage
- ✅ Kushughulikia utendaji wa kutiririsha kwa matukio
- ✅ Kuunda watendaji maalum na @handler
- ✅ Kuongoza mtiririko wa kazi kulingana na maamuzi ya binadamu
- ✅ Kujenga mawakala wa AI wa kuingiliana wanaoshirikiana na binadamu

**Hii ni mfumo muhimu wa kujenga mifumo ya AI inayoweza kuaminika na kudhibitiwa!** 🚀



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kwa usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya asili inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
